# Report: Smartcab Project
*by Oliver Tacke (October 30, 2016)*

## Implement a Basic Driving Agent
To begin, your only task is to get the **smartcab** to move around in the environment. At this point, you will not be concerned with any sort of optimal driving policy. Note that the driving agent is given the following information at each intersection:

- The next waypoint location relative to its current location and heading.
- The state of the traffic light at the intersection and the presence of oncoming vehicles from other directions.
- The current time left from the allotted deadline.

To complete this task, simply have your driving agent choose a random action from the set of possible actions (`None, 'forward', 'left', 'right'`) at each intersection, disregarding the input information above. Set the simulation deadline enforcement, `enforce_deadline` to `False` and observe how it performs.

---

### Question
*Observe what you see with the agent's behavior as it takes random actions. Does the **smartcab** eventually make it to the destination? Are there any other interesting observations to note?*

### ANSWER
As we would expect, the agent showed completely erratic behavior. Still, even with random actions represented by $\alpha = 1$, $\gamma = 0$ and $\epsilon = 1$ averaged over 100 runs each with 100 training trials (`n_times=100`), the agent can reach its destination quite often. In its best run it fails in only about 33 cases by hitting the hard time limit (-100):
```
RESULTS FOR 100 RUN(S) WITH 100 TRIAL(S) EACH
Highest success rate is 0.6664 for
  alpha=1,
  gamma=0, and
  epsilon=1.
with average traffic violations per cab ride: 21.33
with total net reward over all 100 trial(s): -279.35
with average moves above optimum per cab ride: 73.84
```
We should also note the high average number of traffic violations per cab ride: 21.33. Good for a Hollywood movie, but probably not for a smart cab. And the violations didn't even pay off, because our total reward after all the trials is negative.

Anyway, given $8 \times 6 = 48$ intersections and the option to do nothing, I expected the cab to fail more often.

Naturally, the results deteriorate if we set `enforce_deadline` to `True`. The cab would reach the destination in about 20% of the cases at best:
```
RESULTS FOR 100 RUN(S) WITH 100 TRIAL(S) EACH
Highest success rate is 0.1978 for
  alpha=1,
  gamma=0, and
  epsilon=1.
with average traffic violations per cab ride: 7.45
with total net reward over all 100 trial(s): 17.43
with average moves above optimum per cab ride: 21.79  
```  
Judging by the other data that I checked, we can see that we're doing our cab a favor with pulling it off the street.

## Inform the Driving Agent
Now that your driving agent is capable of moving around in the environment, your next task is to identify a set of states that are appropriate for modeling the **smartcab** and environment. The main source of state variables are the current inputs at the intersection, but not all may require representation. You may choose to explicitly define states, or use some combination of inputs as an implicit state. At each time step, process the inputs and update the agent's current state using the `self.state` variable. Continue with the simulation deadline enforcement `enforce_deadline` being set to `False`, and observe how your driving agent now reports the change in state as the simulation progresses.

---

### QUESTION
*What states have you identified that are appropriate for modeling the **smartcab** and environment? Why do you believe each of these states to be appropriate for this problem?*

### ANSWER
Naively, we could use all the `inputs` containing information about the light and possible cars and their directions, the next `waypoint`, and also the `deadline` to constitute states. Since `deadline` can take up lots of different values, it would dramatically increase the number of states and thus training time. If we'd like to increase the size of the grid later, we'd even be in a worse situation.

Of course, we could also think of something more complicated like $\left \lfloor{log_{3}({deadline})}\right \rfloor$. This approach would not only reduce the number of states resulting by changes in `deadline`, but we could also tackle the importance of the remaining time: the state would change more often the smaller the remaining time becomes. Anyway, this is should also be possible by adjusting $\alpha$, $\gamma$, and $\epsilon$ according to the `deadline`.

The next best thing would be 5 variables with 2 or 4 possible values:

- $light \in \{red, green\}$
- $left \in \{None, forward, left, right\}$
- $oncoming \in \{None, forward, left, right\}$
- $right \in \{None, forward, left, right\}$
- $waypoint \in \{None, forward, left, right\}$

Having a closer look at the traffic laws given for this project, we can see that `right` doesn't give us important information:
- If there's a green light (given that other cars obey the traffic rules), we'd only have to check for oncoming traffic that might cross our path if we're turing left.
- If there's a red light and we go left or forward, then we will definitely be punished regardless of other cars.
- If there's a red light and we turn right, then only cars from our left would be relevant.

In conclusion, `right` is redundant and we can stick with `light`, `left`, `oncoming` and `waypoint`.

### OPTIONAL
*How many states in total exist for the **smartcab** in this environment? Does this number seem reasonable given that the goal of Q-Learning is to learn and make informed decisions about each state? Why or why not?*

### ANSWER
The approach taken using `light`, `left`, `oncoming` and `waypoint` generates $2 \times 4 \times 4 \times 4 = 128$ different states possible. This will allow the agent to make good decisions, but it will also learn quite slowly. For Q-Learning, there are many states that have to be visited multiple times in order to learn the value of each action possible, so we must also take these into account, resulting in 512 different fields for our Q table.

Still, given the 100 trials set for this assignment, it should work quite well. The variable `light` has only to possible values but gives us a lot of indicators for driving well. Combining it with `waypoint` with 4 different values for moving quickly will also give us good directions. They're our "principal components" that will do most of the work within the algorithm. For our environment with only three other cars, we could probably even get away with only these using 8 states thus learning quickly, but with more cars the probability of crashes would increase and we'd probably want to avoid these at all costs (at least in Germany which seems to become technophobe judging by the reactions after the resent accient by a Tesla car).

The most extreme alternative (besides driving randomly) would probably be to ignore everything but `waypoint`. We'd only have 4 states, and given the small amount of traffic this might even work - but I'd probably not use such a smartcab in real life, and we better don't come across a cop...

## Implement a Q-Learning Driving Agent

With your driving agent being capable of interpreting the input information and having a mapping of environmental states, your next task is to implement the Q-Learning algorithm for your driving agent to choose the best action at each time step, based on the Q-values for the current state and action. Each action taken by the **smartcab** will produce a reward which depends on the state of the environment. The Q-Learning driving agent will need to consider these rewards when updating the Q-values. Once implemented, set the simulation deadline enforcement `enforce_deadline` to `True`. Run the simulation and observe how the **smartcab** moves about the environment in each trial.

The formulas for updating Q-values can be [found in this video](https://classroom.udacity.com/nanodegrees/nd009/parts/0091345409/modules/e64f9a65-fdb5-4e60-81a9-72813beebb7e/lessons/5446820041/concepts/6348990570923).

---

### QUESTION
*What changes do you notice in the agent's behavior when compared to the basic driving agent when random actions were always taken? Why is this behavior occurring?*

### ANSWER
To me it is not completely clear how $\alpha$, $\gamma$ and $\epsilon$ should be set for this first run, so I simply used `random.random()` to generate random values for each of them. I also had a look at the results before setting `enforce_deadline` to `True`. In most cases, even with these random values the performance increased dramatically. In some cases, the agent wouldn't fail anymore. That was to be expected, because now the agent learns over time and uses the rewards as guidance for his actions instead of driving randomly. Furthermore, the agent violates traffic rules less frequently and moves to the destination quicker. Learning is also accountable for this, because the agent will receive large negative feedback for violating rules and also negative feedback for deviating from the next waypoint suggested by the planner. I also noticed that the agent tended to only fail in eary trials if at all. That's because it takes some time to fill in all the "Q values" for a state, but after that's done: boom. We could achieve the same with increasing the number of trials, of course, which would give the algorithm more time for exploring all possible states and computing the utilty for those depending on the action we'd take.

As an example, here's just one result for different random combinations of $\alpha$, $\gamma$ and $\epsilon$ also averaged over 100 runs:
```
RESULTS FOR 100 RUNS WITH 100 TRIALS EACH
Highest success rate is 0.9889 for
  alpha=0.813522964366,
  gamma=0.699908332339, and
  epsilon=0.442052994264.
with average traffic violations per cab ride: 4.13
with total net reward over all 100 trials: 2377.79
with average moves above optimum per cab ride: 26.74
```   
Also, I ran 100 runs with a decaying value for $\alpha$ and for $\epsilon$, and again with a random value for $\gamma$. It resulted in:

```
RESULTS FOR 100 RUN(S) WITH 100 TRIAL(S) EACH
Highest success rate is 0.955 for
  alpha=-1,
  gamma=0.625087255681, and
  epsilon=-1.
with average traffic violations per cab ride: 2.21
with total net reward over all 100 trial(s): 2484.12
with average moves above optimum per cab ride: 19.78
```
After these trials, I set `enforce_deadline` to `True` and simply set $\alpha$ and $\gamma$ to 0.5 and $\epsilon$ to 0.25, so the agent would still randomly cruise the streets in about 25 % of all moves, moderately consider previous visits and moderately take into account the utility of the next state. In 100 runs with 100 iterations each, the average success rate was about 79 % - still better than the random approach even though now we have a deadline:
```
RESULTS FOR 100 RUN(S) WITH 100 TRIAL(S) EACH
Highest success rate is 0.7903 for
  alpha=0.5,
  gamma=0.5, and
  epsilon=0.25.
with average traffic violations per cab ride: 1.52
with total net reward over all 100 trial(s): 2172.55
with average moves above optimum per cab ride: 12.85
```

## Improve the Q-Learning Driving Agent
Your final task for this project is to enhance your driving agent so that, after sufficient training, the **smartcab** is able to reach the destination within the allotted time safely and efficiently. Parameters in the Q-Learning algorithm, such as the learning rate (`alpha`), the discount factor (`gamma`) and the exploration rate (`epsilon`) all contribute to the driving agent’s ability to learn the best action for each state. To improve on the success of your **smartcab**:

- Set the number of trials, `n_trials`, in the simulation to 100.
- Run the simulation with the deadline enforcement `enforce_deadline` set to `True` (you will need to reduce the update delay `update_delay` and set the `display` to `False`).
- Observe the driving agent’s learning and **smartcab’s** success rate, particularly during the later trials.
- Adjust one or several of the above parameters and iterate this process.
- This task is complete once you have arrived at what you determine is the best combination of parameters required for your driving agent to learn successfully.

---

### QUESTION
*Report the different values for the parameters tuned in your basic implementation of Q-Learning. For which set of parameters does the agent perform best? How well does the final driving agent perform?*

### ANSWER
In order to find the best combination of $\alpha$, $\gamma$ and $\epsilon$, I quickly implemented a grid search. I lazily started it with
- a set of $\{0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, -1\}$ for $\alpha$, where -1 indicates $\alpha$-decay using $\alpha = \frac{1}{t}$,
- a set of $\{0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9\}$ for $\gamma$,
- a set  of $\{0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, -1\}$ for $\epsilon$, where -1 means setting a decay via $\epsilon = \frac{1}{t}$, and
- then averaged the success rate of 100 runs for each combination, each with 100 trials for training (`n_times=100`).
A feasible problem like ours can still be tackled fairly well this way ;-) After quite some time of waiting, the result was:
```
RESULTS FOR 100 RUN(S) WITH 100 TRIAL(S) EACH
Highest success rate is 0.8345 for
  alpha=0.1,
  gamma=0.1, and
  epsilon=0.2.
with average traffic violations per cab ride: 1.55
with total net reward over all 100 trial(s): 2005.70
with average moves above optimum per cab ride: 11.71
```
Afterwards, I explored the space around this solution a little more diligently by setting $\alpha \in \{0.03, 0.07, 0.1, 0.13, 0.17\}$, $\gamma \in \{0.03, 0.07, 0.1, 0.13, 0.17\}$ and $\epsilon \in \{0.13, 0.17, 0.2, 0.23, 0.27\}$. The final result was:
```
RESULTS FOR 100 RUN(S) WITH 100 TRIAL(S) EACH
Highest success rate is 0.839 for
  alpha=0.13,
  gamma=0.13, and
  epsilon=0.23.
with average traffic violations per cab ride: 1.74
with total net reward over all 100 trial(s): 2033.73
with average moves above optimum per cab ride: 12.05

```
Since the results are only slightly better (and there's still some variation in the results even with averaging over 100 runs), "I call it an agent". Not considering the way we're measuring sucess (see final answer for more on that), with $\alpha = 0.13$, $\gamma = 0.13$, and $\epsilon = 0.23$, we have a success rate of roughly 0.84, average traffic violations of 1.74 per cab ride (because of my initial value for the q table) and 12.09 moves above optimum per cab ride.

Would have been much easier to just increase the number of trials some more and use the final q tables, but we were restricted to `n_trials=100`.

### QUESTION
*Does your agent get close to finding an optimal policy, i.e. reach the destination in the minimum possible time, and not incur any penalties? How would you describe an optimal policy for this problem?*

### ANSWER
An optimal policy could be "Safety first, but then be quick!", and I think the agent is pretty close to that. It could need some more training than "only" 100 trials, but the states chosen will guarantee that following a "safety first" mindset can be achieved. To make sure about that, $\epsilon$ should be set to 0 either after a certain number of trials or by using a function that converges to 0 instead of a fixed value as I tried (but it seems that this would have to be tunes, see results above). This way, there really wouldn't be any random driving after a certain amount of time, but the agent would solely rely on the q tables from the rewards it got. We should probably use this agent "version" to judge the success rate.

One could argue that the average penalty per cab ride is really small but not zero. However, a closer look at the data tells us that those penalties primarily occur in early trials when the agent is still learning and sometimes later on when he chooses to explore state space by a random move. This could also be prevented by letting $\epsilon$ converge to zero after a certain amount of time. Again, we should probably use the fully trained agent to judge the number of penalties per cab ride.

Also, one might argue that it is not yet optimal because the number of moves the agent needs still exceed the distance between two intersections, but we must consider red lights that force us to wait or use a detour. In consequence, reaching the target in minimum time is really only possible if there's no red light, so this is hardly possible.

I found that even an agent perfectly obeying the traffic rules would fail sometimes because of bad luck with the traffic lights. Interesting... But we probably wouldn't want to risk ignoring a red light even when time is short and there's no traffic, would we?

We're close to the end now... ;-) If we use the best combination of parameters that we have found so far for our best solution (see Appendix A), let the agent train in 1 run with 100 trials for training that yields really good results (mind the variation), then store the q table as default, and finally set $\epsilon$ to 0, we should now be measuring the "final" performance after 100 trials and it should be good:
```
RESULTS FOR 1 RUN(S) WITH 100 TRIAL(S) EACH
Highest success rate is 1.0 for
  alpha=0.13,
  gamma=0.13, and
  epsilon=0.
with average traffic violations per cab ride: 0.04
with total net reward over all 100 trial(s): 2223.00
with average moves above optimum per cab ride: 7.84
```
Pretty neat smartcab! Still, it could fail sometimes... Just increase `n_runs` and see for yourself... Bad luck with traffic lights can mess up your ride.

## Appendix A (q table of best solution)
This is the q table that was found using alpha=0.1, gamma=0.1, epsilon=0.1, and n_times=100.

It's a dictionary that uses a tupel of a state (tupel of `light`, `oncoming`, `left`, and `waypoint`) and an action as key.

```
{
(('red', None, None, 'right'), 'left'): -0.3024206389545996,
(('red', None, 'right', 'right'), None): 5,
(('green', 'left', None, 'right'), None): 3.93295805,
(('red', None, 'left', 'forward'), None): 3.4881404945449996,
(('green', None, None, 'right'), None): 0.8555011249117009,
(('red', 'left', None, 'right'), None): 3.4881404945449996,
(('red', None, None, 'forward'), 'forward'): -0.9955673115563245,
(('red', 'right', None, 'forward'), None): 5,
(('green', None, None, 'right'), 'right'): 3.0090979077212117,
(('green', None, 'left', 'forward'), None): 2.1582044745758604,
(('green', None, 'right', 'left'), None): 5,
(('green', None, 'forward', 'right'), None): 3.93295805,
(('green', 'right', None, 'forward'), None): 4.4345,
(('red', 'left', None, 'forward'), 'forward'): 2.65538624245379,
(('red', None, None, 'forward'), 'right'): -0.25196171465387174,
(('red', None, 'forward', 'forward'), None): 4.4345,
(('red', None, None, 'left'), 'forward'): -0.7173040224973123,
(('green', None, 'forward', 'forward'), None): 2.1582044745758604,
(('red', None, 'right', 'forward'), 'right'): 5,
(('green', 'right', None, 'right'), None): 5,
(('red', 'left', None, 'forward'), 'right'): 2.4113488161286427,
(('green', None, 'forward', 'left'), None): 3.09363180461196,
(('red', 'left', None, 'forward'), None): 2.7437420475103473,
(('red', None, None, 'right'), None): 0.6794344147737408,
(('red', None, 'forward', 'left'), None): 5,
(('red', None, 'left', 'right'), None): 5,
(('green', None, None, 'forward'), 'left'): -0.17142315293395632,
(('green', None, None, 'right'), 'forward'): 0.7282309446845889,
(('green', 'left', None, 'forward'), None): 1.9141115485013305,
(('green', None, None, 'left'), None): 1.5407409641532746,
(('red', None, None, 'left'), None): 9.984997195891648e-05,
(('red', None, None, 'left'), 'right'): -0.04633182005315557,
(('red', None, None, 'left'), 'left'): -0.501888914801135,
(('red', None, None, 'right'), 'right'): 3.1816836514669364,
(('green', 'forward', None, 'forward'), None): 4.4345,
(('green', None, 'left', 'forward'), 'right'): 2.650936855,
(('red', 'right', None, 'right'), None): 5,
(('green', None, None, 'forward'), None): 0.5194278303776698,
(('green', None, None, 'forward'), 'forward'): 5.06338558002154,
(('green', 'right', None, 'left'), None): 4.4345,
(('green', None, 'forward', 'forward'), 'right'): 3.7453621250134295,
(('green', None, None, 'forward'), 'right'): 0.12841901587686397,
(('red', 'forward', None, 'left'), None): 3.93295805,
(('green', None, 'right', 'right'), None): 5,
(('red', None, None, 'right'), 'forward'): -0.2664737698823249,
(('green', None, None, 'right'), 'left'): 0.9872779719359829,
(('green', None, None, 'left'), 'forward'): 0.7897815099111861,
(('red', 'left', 'right', 'forward'), None): 5,
(('red', None, 'forward', 'right'), 'right'): 5,
(('red', None, 'right', 'left'), None): 5,
(('green', None, 'forward', 'forward'), 'forward'): 5,
(('green', None, None, 'left'), 'left'): 2.493319971504075,
(('red', 'right', None, 'forward'), 'right'): 5,
(('green', 'left', None, 'left'), None): 5,
(('red', None, None, 'forward'), 'left'): -0.9930697446003097,
(('green', 'left', None, 'right'), 'left'): 3.7658360799273543,
(('red', 'forward', None, 'left'), 'forward'): 3.6876610499999996,
(('green', None, None, 'left'), 'right'): 0.7449793641663388,
(('red', None, None, 'forward'), None): 1.256324494036063e-24,
(('green', None, 'left', 'right'), None): 4.4345,
(('green', None, 'forward', 'right'), 'right'): 4.092817945776977,
(('green', None, 'left', 'right'), 'right'): 5
}
```